In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper, CategoricalImputer
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error

In [2]:
#import modelos
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import ExtraTreeRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor

# Procesamiento de los datos

In [3]:
#filtrando los datos
df_train= pd.read_csv('train.csv', dtype={
    'tipodepropiedad':'category', 'ciudad':'category',\
    'provincia':'category'}, parse_dates=[16])

df_test = pd.read_csv('test.csv', dtype={
    'tipodepropiedad':'category', 'ciudad':'category',\
    'provincia':'category'}, parse_dates=[16])

### Manejo de valores nulos

In [4]:
df_train.isnull().sum()

id                                 0
titulo                          5387
descripcion                     1619
tipodepropiedad                   46
direccion                      53072
ciudad                           372
provincia                        155
antiguedad                     43555
habitaciones                   22471
garages                        37765
banos                          26221
metroscubiertos                17400
metrostotales                  51467
idzona                         28621
lat                           123488
lng                           123488
fecha                              0
gimnasio                           0
usosmultiples                      0
piscina                            0
escuelascercanas                   0
centroscomercialescercanos         0
precio                             0
dtype: int64

A los datos del dataframe que tienen una de las columnas (de 'metrostotales' y 'metroscubiertos') pero no la otra, copiamos el dato de una a la otra según sea necesario.

In [5]:
## funcion para setear los superficies
def set_metros(row):
    total = row.metrostotales
    covered = row.metroscubiertos
    if np.isnan(total):
        row.metrostotales = covered
        return row
    if np.isnan(covered):
        row.metroscubiertos = total
        return row
    return row

In [6]:
df_train[['metrostotales', 'metroscubiertos']] = df_train[['metrostotales', 'metroscubiertos']].apply(set_metros, axis = 1)
df_test[['metrostotales', 'metroscubiertos']] = df_test[['metrostotales', 'metroscubiertos']].apply(set_metros, axis = 1)

In [7]:
df_train = df_train.loc[~df_train['tipodepropiedad'].isnull()] #Elimino los datos que no tengan tipo de propiedad (46)
df_train['garages'] = df_train['garages'].fillna(value = 0)


In [8]:
df_test['garages'] = df_test['garages'].fillna(value = 0)

In [9]:
num_cols_con_nulls = ['antiguedad', 'habitaciones', 'banos', 'idzona']
cat_cols_con_nulls = ['ciudad', 'provincia']

In [10]:
# define numerical imputer
num_imputer = SimpleImputer(strategy='mean')

# imputing on numerical data
df_train[num_cols_con_nulls] = num_imputer.fit_transform(df_train[num_cols_con_nulls])
df_test[num_cols_con_nulls] = num_imputer.fit_transform(df_test[num_cols_con_nulls])

In [11]:
from sklearn.base import TransformerMixin

class SeriesImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        If the Series is of dtype Object, then impute with the most frequent object.
        If the Series is not of dtype Object, then impute with the mean.  

        """
    def fit(self, X, y=None):
        self.fill = X.value_counts().index[0]
        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)

In [12]:
# define categorical imputer
cat_imputer = SeriesImputer()

# imputing on categorical data
df_train['ciudad'] = cat_imputer.fit_transform(df_train['ciudad'])
df_train['provincia'] = cat_imputer.fit_transform(df_train['provincia'])

df_test['ciudad'] = cat_imputer.fit_transform(df_test['ciudad'])
df_test['provincia'] = cat_imputer.fit_transform(df_test['provincia'])
df_test['idzona'] = cat_imputer.fit_transform(df_test['idzona'])
df_test.isnull().sum()

id                                0
titulo                         1378
descripcion                     401
tipodepropiedad                   7
direccion                     13191
ciudad                            0
provincia                         0
antiguedad                        0
habitaciones                      0
garages                           0
banos                             0
metroscubiertos                   0
metrostotales                     0
idzona                            0
lat                           30695
lng                           30695
fecha                             0
gimnasio                          0
usosmultiples                     0
piscina                           0
escuelascercanas                  0
centroscomercialescercanos        0
dtype: int64

In [13]:
df_train.isnull().sum()

id                                 0
titulo                          5387
descripcion                     1615
tipodepropiedad                    0
direccion                      53028
ciudad                             0
provincia                          0
antiguedad                         0
habitaciones                       0
garages                            0
banos                              0
metroscubiertos                    0
metrostotales                      0
idzona                             0
lat                           123457
lng                           123457
fecha                              0
gimnasio                           0
usosmultiples                      0
piscina                            0
escuelascercanas                   0
centroscomercialescercanos         0
precio                             0
dtype: int64

## Encoding variables categóricas

### ID_Zona

Efectuare un ordinal encoding, teniendo en cuenta los precios promedio por id_zona obtenidos a partir del TP1

In [14]:
df_train.groupby('idzona').agg({'tipodepropiedad' : 'count'}).mean()

tipodepropiedad    12.168053
dtype: float64

In [15]:
#Agrego precio por metro cuadrado
df_train['precioporm2'] = df_train['precio'] / df_train['metrostotales']

In [16]:
precio_m2_por_idzona = df_train.groupby('idzona').agg({'precioporm2' : 'mean'})
precio_m2_por_idzona.reset_index(inplace = True)
precio_m2_por_idzona = precio_m2_por_idzona.sort_values(by = 'precioporm2')
precio_m2_por_idzona

,idzona,precioporm2
18431,129693.0,720.000000
17020,114109.0,781.093596
5348,41576.0,785.714286
13186,80278.0,800.000000
6789,50076.0,800.000000
17094,114292.0,800.000000
15505,99326.0,840.000000
5467,42365.0,859.950860
13224,81747.0,875.000000
17220,115099.0,875.000000


In [17]:
def cambiar_valores(x):
    i = 0
    if x.name == 'precioporm2':
        i+=1
        return i
    return x

precio_m2_por_idzona = precio_m2_por_idzona.apply(cambiar_valores)
precio_m2_por_idzona['precioporm2'] = precio_m2_por_idzona['precioporm2'].cumsum()
precio_m2_por_idzona

,idzona,precioporm2
18431,129693.0,1
17020,114109.0,2
5348,41576.0,3
13186,80278.0,4
6789,50076.0,5
17094,114292.0,6
15505,99326.0,7
5467,42365.0,8
13224,81747.0,9
17220,115099.0,10


In [18]:
orden_idzona = precio_m2_por_idzona.set_index('idzona').to_dict()
orden_idzona = orden_idzona['precioporm2']

In [19]:
df_train['idzona_ordinal'] = df_train.idzona.map(orden_idzona)
df_test['idzona_ordinal'] = df_test.idzona.map(orden_idzona)
df_test['idzona_ordinal'] = cat_imputer.fit_transform(df_test['idzona_ordinal'])
df_test.isnull().sum()

id                                0
titulo                         1378
descripcion                     401
tipodepropiedad                   7
direccion                     13191
ciudad                            0
provincia                         0
antiguedad                        0
habitaciones                      0
garages                           0
banos                             0
metroscubiertos                   0
metrostotales                     0
idzona                            0
lat                           30695
lng                           30695
fecha                             0
gimnasio                          0
usosmultiples                     0
piscina                           0
escuelascercanas                  0
centroscomercialescercanos        0
idzona_ordinal                    0
dtype: int64

### Tipo de propiedad

In [20]:
df_train['tipodepropiedad'].unique()

[Apartamento, Casa en condominio, Casa, Terreno, Terreno comercial, ..., Terreno industrial, Huerta, Lote, Hospedaje, Garage]
Length: 24
Categories (24, object): [Apartamento, Casa en condominio, Casa, Terreno, ..., Huerta, Lote, Hospedaje, Garage]

In [21]:
df_train.shape

(239954, 25)

In [22]:
#One Hot Encoding
df_train = pd.get_dummies(df_train, prefix = ['Tipo'], columns = ['tipodepropiedad'])
df_test = pd.get_dummies(df_test, prefix = ['Tipo'], columns = ['tipodepropiedad'])

In [23]:
df_train.shape

(239954, 48)

In [24]:
df_train.head()

,id,titulo,descripcion,direccion,ciudad,provincia,antiguedad,habitaciones,garages,banos,...,Tipo_Oficina comercial,Tipo_Otros,Tipo_Quinta Vacacional,Tipo_Rancho,Tipo_Terreno,Tipo_Terreno comercial,Tipo_Terreno industrial,Tipo_Villa,Tipo_Hospedaje,Tipo_Garage
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,8.116273,2.0,1.0,2.0,...,0,0,0,0,0,0,0,0,0,0
1,53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.000000,3.0,2.0,2.0,...,0,0,0,0,0,0,0,0,0,0
2,247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Urbi Tonala,Tonalá,Jalisco,5.000000,3.0,2.0,2.0,...,0,0,0,0,0,0,0,0,0,0
3,209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.000000,2.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
4,185997,paseos del sol,bonito departamento en excelentes condiciones ...,PASEOS DEL SOL,Zapopan,Jalisco,10.000000,2.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0


### Provincia

Efectuare un ordinal encoding, teniendo en cuenta los precios promedio por provincia obtenidos a partir del TP1

In [25]:
# precio_promedio_por_provincia = pd.read_csv('provincias_por_precio')
# precio_promedio_por_provincia = precio_promedio_por_provincia.sort_values(by = 'precio_por_m2_mean')



# def cambiar_valores(x):
#     i = 0
#     if x.name == 'precio_por_m2_mean':
#         i+=1
#         return i
#     return x

# precio_promedio_por_provincia = precio_promedio_por_provincia.apply(cambiar_valores)
# precio_promedio_por_provincia['precio_por_m2_mean'] = precio_promedio_por_provincia['precio_por_m2_mean'].cumsum()
# precio_promedio_por_provincia

In [26]:
# orden_provincias = precio_promedio_por_provincia.set_index('provincia').to_dict()
# orden_provincias = orden_provincias['precio_por_m2_mean']
# orden_provincias

In [27]:
# df['provincia'] = df['provincia'].replace({'Edo. de México' : 'Mexico',
#                                                'Querétaro' : 'Queretaro',
#                                                'Nuevo León' : 'Nuevo Leon',
#                                                'San luis Potosí' : 'San Luis Potosi',
#                                                'Yucatán' : 'Yucatan',
#                                                'Baja California Norte' : 'Baja California',
#                                                'Michoacán' : 'Michoacan'})
# df_test['provincia'] = df_test['provincia'].replace({'Edo. de México' : 'Mexico',
#                                                'Querétaro' : 'Queretaro',
#                                                'Nuevo León' : 'Nuevo Leon',
#                                                'San luis Potosí' : 'San Luis Potosi',
#                                                'Yucatán' : 'Yucatan',
#                                                'Baja California Norte' : 'Baja California',
#                                                'Michoacán' : 'Michoacan'})

In [28]:
# df['provincia_ordinal'] = df.provincia.map(orden_provincias)
# df_test['provincia_ordinal'] = df.provincia.map(orden_provincias)

df_train.head()

,id,titulo,descripcion,direccion,ciudad,provincia,antiguedad,habitaciones,garages,banos,...,Tipo_Oficina comercial,Tipo_Otros,Tipo_Quinta Vacacional,Tipo_Rancho,Tipo_Terreno,Tipo_Terreno comercial,Tipo_Terreno industrial,Tipo_Villa,Tipo_Hospedaje,Tipo_Garage
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,8.116273,2.0,1.0,2.0,...,0,0,0,0,0,0,0,0,0,0
1,53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.000000,3.0,2.0,2.0,...,0,0,0,0,0,0,0,0,0,0
2,247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Urbi Tonala,Tonalá,Jalisco,5.000000,3.0,2.0,2.0,...,0,0,0,0,0,0,0,0,0,0
3,209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.000000,2.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
4,185997,paseos del sol,bonito departamento en excelentes condiciones ...,PASEOS DEL SOL,Zapopan,Jalisco,10.000000,2.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
#df['ciudad'] = df['ciudad'].cat.codes
#df['provincia'] = df['provincia'].cat.codes
# df_train_drop = df_train.drop(columns = ['provincia', 'ciudad'])
# df_test_drop = df_test.drop(columns = ['provincia', 'ciudad'])

In [30]:
df_train.isnull().sum()

id                                         0
titulo                                  5387
descripcion                             1615
direccion                              53028
ciudad                                     0
provincia                                  0
antiguedad                                 0
habitaciones                               0
garages                                    0
banos                                      0
metroscubiertos                            0
metrostotales                              0
idzona                                     0
lat                                   123457
lng                                   123457
fecha                                      0
gimnasio                                   0
usosmultiples                              0
piscina                                    0
escuelascercanas                           0
centroscomercialescercanos                 0
precio                                     0
precioporm

In [31]:
df_train['fecha'] = pd.to_datetime(df_train['fecha'])

y = df_train['precio'].values

df_train['dia'] = df_train['fecha'].dt.day
df_train['mes'] = df_train['fecha'].dt.month
df_train['anio'] = df_train['fecha'].dt.year

df_train['cant_extras'] = sum([df_train['gimnasio'],df_train['usosmultiples'],df_train['piscina'],
                        df_train['escuelascercanas'],df_train['centroscomercialescercanos']])
df_train_fin = df_train.drop(columns=['fecha', 'id', 'idzona', 'titulo', 'descripcion', 'direccion', 'lat', 'lng', 'precio',\
                                      'provincia', 'ciudad','gimnasio','usosmultiples','piscina','escuelascercanas',\
                                      'centroscomercialescercanos','dia','metroscubiertos','precioporm2','Tipo_Hospedaje','Tipo_Garage']).copy()
df_train_fin.isnull().sum()

antiguedad                            0
habitaciones                          0
garages                               0
banos                                 0
metrostotales                         0
idzona_ordinal                        0
Tipo_Apartamento                      0
Tipo_Bodega comercial                 0
Tipo_Casa                             0
Tipo_Casa en condominio               0
Tipo_Casa uso de suelo                0
Tipo_Departamento Compartido          0
Tipo_Duplex                           0
Tipo_Edificio                         0
Tipo_Huerta                           0
Tipo_Inmuebles productivos urbanos    0
Tipo_Local Comercial                  0
Tipo_Local en centro comercial        0
Tipo_Lote                             0
Tipo_Nave industrial                  0
Tipo_Oficina comercial                0
Tipo_Otros                            0
Tipo_Quinta Vacacional                0
Tipo_Rancho                           0
Tipo_Terreno                          0


In [32]:
df_test['fecha'] = pd.to_datetime(df_test['fecha'])

df_test['dia'] = df_test['fecha'].dt.day
df_test['mes'] = df_test['fecha'].dt.month
df_test['anio'] = df_test['fecha'].dt.year

# df_test = df_test.drop(columns=['fecha', 'id', 'idzona', 'titulo', 'descripcion', 'direccion', 'lat', 'lng']).copy()


df_test['cant_extras'] = sum([df_test['gimnasio'],df_test['usosmultiples'],df_test['piscina'],
                        df_test['escuelascercanas'],df_test['centroscomercialescercanos']])

df_test_fin = df_test.drop(columns=['fecha', 'id', 'idzona', 'titulo', 'descripcion', 'direccion', 'lat', 'lng',\
                                      'provincia', 'ciudad','gimnasio','usosmultiples','piscina','escuelascercanas',\
                                      'centroscomercialescercanos','dia','metroscubiertos']).copy()

df_test_fin.isnull().sum()

antiguedad                            0
habitaciones                          0
garages                               0
banos                                 0
metrostotales                         0
idzona_ordinal                        0
Tipo_Apartamento                      0
Tipo_Bodega comercial                 0
Tipo_Casa                             0
Tipo_Casa en condominio               0
Tipo_Casa uso de suelo                0
Tipo_Departamento Compartido          0
Tipo_Duplex                           0
Tipo_Edificio                         0
Tipo_Inmuebles productivos urbanos    0
Tipo_Local Comercial                  0
Tipo_Local en centro comercial        0
Tipo_Nave industrial                  0
Tipo_Oficina comercial                0
Tipo_Otros                            0
Tipo_Quinta Vacacional                0
Tipo_Rancho                           0
Tipo_Terreno                          0
Tipo_Terreno comercial                0
Tipo_Terreno industrial               0


## Ejecucion de modelos

In [33]:
X = df_train_fin.copy().values
y_log = np.log(y)

#splitear train y test
X_train, X_test, y_train, y_test = train_test_split(X, y_log, test_size=0.25, random_state=42)

test_entrega = df_test_fin.copy().values

In [34]:
#randomforest
random_forest = RandomForestRegressor(n_estimators=200, max_depth=500, n_jobs=-1)
random_forest.fit(X_train, y_train)
precio_pred=random_forest.predict(X_test)
mean_absolute_error(np.exp(precio_pred), np.exp(y_test))

598205.27077212033

In [35]:
# precio_pred=random_forest.predict(test_entrega)

# LO QUE SIGUE ES PARA PROBAR TODOS LOS MODELOS!

In [36]:
def ejecutar_modelos(X_train, y_train, X_test, y_test, modelos, nombres):
    resultado = []
    for nombre,modelo in zip(modelos_nombre, modelos):
        print('------------------------------------------------')
        print('Comienza a entrenar: '+ nombre)
        print()
        modelo = modelo   
        modelo.fit(X_train,y_train)
        score = modelo.score(X_test,y_test)
        resultado.append((str(nombre), str(score)))
        print('**FINALIZADO** ' + nombre + ' Score: ' + str(score))
        print()
    return resultado

In [37]:
#splitear train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [38]:
modelos=[LinearRegression(),KNeighborsRegressor(),Ridge(),Lasso(),MLPRegressor(alpha=20),DecisionTreeRegressor(),ExtraTreeRegressor(),XGBRegressor(),RandomForestRegressor(),AdaBoostRegressor(),GradientBoostingRegressor(),BaggingRegressor()]
modelos_nombre=['LinearRegression','KNNRegressor','Ridge','Lasso','MLPRegressor','DecisionTree','ExtraTree','XGBoost','RandomForest','AdaBoost','GradientBoost','Bagging']
score_=[]

In [39]:
resultado_normal = ejecutar_modelos(X_train, y_train, X_test, y_test, modelos, modelos_nombre)

------------------------------------------------
Comienza a entrenar: LinearRegression

**FINALIZADO** LinearRegression Score: 0.575098079671

------------------------------------------------
Comienza a entrenar: KNNRegressor

**FINALIZADO** KNNRegressor Score: 0.658969886638

------------------------------------------------
Comienza a entrenar: Ridge

**FINALIZADO** Ridge Score: 0.575085045284

------------------------------------------------
Comienza a entrenar: Lasso

**FINALIZADO** Lasso Score: 0.575095438343

------------------------------------------------
Comienza a entrenar: MLPRegressor

**FINALIZADO** MLPRegressor Score: 0.672866743123

------------------------------------------------
Comienza a entrenar: DecisionTree

**FINALIZADO** DecisionTree Score: 0.55863996612

------------------------------------------------
Comienza a entrenar: ExtraTree

**FINALIZADO** ExtraTree Score: 0.534026263964

------------------------------------------------
Comienza a entrenar: XGBoost

[19

In [40]:
#aplico logaritmo
y_log = np.log(y)
X_train, X_test, y_train_log, y_test_log = train_test_split(X, y_log, test_size=0.25, random_state=42)

In [41]:
resultado_log = ejecutar_modelos(X_train, y_train_log, X_test, y_test_log, modelos, modelos_nombre)

------------------------------------------------
Comienza a entrenar: LinearRegression

**FINALIZADO** LinearRegression Score: 0.738084012651

------------------------------------------------
Comienza a entrenar: KNNRegressor

**FINALIZADO** KNNRegressor Score: 0.731394777376

------------------------------------------------
Comienza a entrenar: Ridge

**FINALIZADO** Ridge Score: 0.738071932237

------------------------------------------------
Comienza a entrenar: Lasso

**FINALIZADO** Lasso Score: 0.653845832862

------------------------------------------------
Comienza a entrenar: MLPRegressor

**FINALIZADO** MLPRegressor Score: -14.7121353843

------------------------------------------------
Comienza a entrenar: DecisionTree

**FINALIZADO** DecisionTree Score: 0.677542539266

------------------------------------------------
Comienza a entrenar: ExtraTree

**FINALIZADO** ExtraTree Score: 0.673685373697

------------------------------------------------
Comienza a entrenar: XGBoost

[1

In [42]:
#estanderizar datos
X_estandar=StandardScaler().fit_transform(X)
y=np.array(y).reshape(-1,1)
y_estandar=StandardScaler().fit_transform(y)
y_estandar=y_estandar.ravel()
X_train_estandar, X_test_estandar, y_train_estandar, y_test_estandar = train_test_split(X_estandar, y_estandar, test_size=0.25, random_state=42)


In [43]:
resultado_estandar = ejecutar_modelos(X_train_estandar, y_train_estandar, X_test_estandar, y_test_estandar, modelos, modelos_nombre)

------------------------------------------------
Comienza a entrenar: LinearRegression

**FINALIZADO** LinearRegression Score: 0.575098079671

------------------------------------------------
Comienza a entrenar: KNNRegressor

**FINALIZADO** KNNRegressor Score: 0.65545476518

------------------------------------------------
Comienza a entrenar: Ridge

**FINALIZADO** Ridge Score: 0.575098029167

------------------------------------------------
Comienza a entrenar: Lasso

**FINALIZADO** Lasso Score: -1.71853319817e-05

------------------------------------------------
Comienza a entrenar: MLPRegressor

**FINALIZADO** MLPRegressor Score: 0.613834744373

------------------------------------------------
Comienza a entrenar: DecisionTree

**FINALIZADO** DecisionTree Score: 0.556007667728

------------------------------------------------
Comienza a entrenar: ExtraTree

**FINALIZADO** ExtraTree Score: 0.536231742435

------------------------------------------------
Comienza a entrenar: XGBoost


In [49]:
#Crea un archivo csv de la forma nombre_del_algoritmo, score

#Columnas: string con nombres de columnas separados por comas
#Resultados: lista de resultados, siendo cada resultado una lista conformada por tuplas (Nombre_algoritmo, Score)
def crear_csv_resultados(columnas, resultados, nombre_archivo):
    with open(nombre_archivo, 'w') as f:
        f.write(columnas + '\n')
        lineas_a_escribir = []
        for i in range(len(resultados[0])):
            linea = resultados[0][i][0] + ','
            for resultado in resultados:
                linea = linea + resultado[i][1] + ','
            linea = linea[:len(linea)-1]
            lineas_a_escribir.append(linea)
        for linea in lineas_a_escribir:
            f.write(linea + '\n')

In [45]:
# NOMBRE_CSV = 'one_hot+ordinal_provincias+ordinal_idzona' # CAMBIARLO ASI NO SE PISAN!!!!
# crear_csv_resultados('nombre,normal,log,estandar',[resultado_normal, resultado_log, resultado_estandar],NOMBRE_CSV)

In [46]:
# pd.read_csv('one_hot_ciudades')

In [47]:
# pd.read_csv('num_imputter_mean')

In [48]:
# pd.read_csv(NOMBRE_CSV)